## Librarby Import

In [5]:
import pandas as pd
import numpy as np

import pymongo

import matplotlib.pyplot as plt
from matplotlib import pyplot,figure

import xgboost as xgb
import lightgbm as lgb
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,AdaBoostRegressor
from sklearn.linear_model import BayesianRidge,Ridge,Lasso,ElasticNet
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.preprocessing import StandardScaler,OneHotEncoder,scale,LabelEncoder

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.decomposition import PCA

from mlxtend.regressor import StackingRegressor
from sklearn.compose import make_column_selector as selector, ColumnTransformer
from skopt import BayesSearchCV

from skopt.space import Real, Categorical, Integer
from scipy.stats import skew

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

import time
start_time = time.time()

In [9]:
# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["stock_data_db"]
collection = db["stock_data"]

# Fetch data from MongoDB
data=collection.find({})

# Convert document based database to pandas dataframe
df = pd.DataFrame(list(data))

# Extract the desired data from the document-converted dataframe
df = pd.DataFrame(df[df['symbol']=='NVDA']['technical_data'][0])

In [10]:
train

,date,open,high,low,close,volume,8EMA,13EMA,144EMA,169EMA,...,close_t-2,close_t-3,Incremental_High,MACD_GOLDEN_CROSS,169EMA_Upper,169EMA_Lower,MACD_Alert,Engulf_Alert,dual_channel_Alert,382_Alert
0,2022-09-22,13.058058,13.136986,12.416645,12.549523,759162000,13.220576,13.610700,18.179119,18.578121,...,13.163962,13.369773,False,False,19.507027,17.649215,0,-1,0,1
1,2022-09-23,12.408652,12.599478,12.245801,12.504564,663309000,13.061462,13.452681,18.100850,18.506667,...,13.248885,13.163962,False,False,19.432001,17.581334,0,-1,0,0
2,2022-09-26,12.479591,12.647438,12.202844,12.216831,547343000,12.873766,13.276131,18.019691,18.432669,...,12.549523,13.248885,False,False,19.354303,17.511036,-1,-1,0,1
3,2022-09-27,12.495575,12.724366,12.246803,12.401661,553854000,12.768854,13.151207,17.942201,18.361716,...,12.504564,12.549523,False,False,19.279802,17.443630,-1,-1,0,0
4,2022-09-28,12.398662,12.811285,12.342713,12.724364,542414000,12.758967,13.090229,17.870231,18.295394,...,12.216831,12.504564,False,True,19.210164,17.380625,-1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2024-09-12,116.839996,120.790001,115.379997,119.139999,367100500,113.285854,114.104462,102.609016,98.776449,...,108.090752,106.460892,True,False,103.715271,93.837626,-1,-1,-1,0
496,2024-09-13,119.080002,119.959999,117.599998,119.099998,238358300,114.577886,114.818110,102.836477,99.015549,...,116.900002,108.090752,False,True,103.966327,94.064772,-1,-1,-1,1
497,2024-09-16,116.790001,118.180000,114.360001,116.779999,248772300,115.067245,115.098380,103.028802,99.224543,...,119.139999,116.900002,False,True,104.185770,94.263316,-1,-1,-1,1
498,2024-09-17,118.169998,118.800003,114.830002,115.589996,231925900,115.183412,115.168611,103.202060,99.417078,...,119.099998,119.139999,False,True,104.387931,94.446224,-1,-1,-1,1


## Check Data Integrity

In [ ]:
#Compute the missing value ratio 
missing_data = ((df.isnull().sum())/len(df))
missing_data = missing_data[missing_data.values > 0].sort_values()

#Visualize the missing value ratio 
fig = plt.figure(figsize=(5,5), dpi = 100)
sn.barplot(x = missing_data.index, y = missing_data.values) 
plt.xticks(rotation=90)
plt.title('Features Missing Ratio')
plt.show()

## Machine Learning Model